# Capítulo 01 - Introdução ao SQL com DuckDB

Este notebook apresenta os conceitos fundamentais de SQL usando DuckDB, um banco de dados analítico in-process de alta performance. Vamos explorar desde a instalação até operações avançadas como junções e agregações.

## Tópicos Abordados:
1. Instalação de Dependências
2. Imports e Configuração Inicial
3. Criação de Tabelas e Estruturas
4. Inserção de Dados (INSERT)
5. Consultas Básicas (SELECT)
6. Expressões e Cálculos
7. Filtros com WHERE
8. Ordenação (ORDER BY)
9. Valores Únicos (DISTINCT)
10. Junções de Tabelas (JOINS)
11. Funções de Agregação
12. Agrupamento (GROUP BY)
13. Filtros em Grupos (HAVING)
14. Atualização de Dados (UPDATE)
15. Exclusão de Dados (DELETE)

## 1. Instalação de Dependências

Primeiro, vamos instalar o DuckDB. O `-q` suprime a saída detalhada da instalação.

In [1]:
%pip install duckdb -q

Note: you may need to restart the kernel to use updated packages.


## 2. Imports e Configuração Inicial

Importando as bibliotecas necessárias:
- `duckdb`: Biblioteca principal para trabalhar com DuckDB
- `os`: Para operações do sistema operacional (se necessário)

In [2]:
# -*- coding: utf-8 -*-
import duckdb
import os

print("✓ Bibliotecas importadas com sucesso!")
print(f"Versão do DuckDB: {duckdb.__version__}")

✓ Bibliotecas importadas com sucesso!
Versão do DuckDB: 1.4.3


## 3. Criação de Tabelas e Estruturas

Vamos criar um banco de dados em memória e duas tabelas de exemplo:
- **weather**: Armazena informações meteorológicas
- **cities**: Armazena informações de localização das cidades

### Banco de Dados em Memória
Usar `:memory:` cria um banco temporário que existe apenas durante a execução do programa.

In [3]:
# Conectar ao banco em memória
con = duckdb.connect(database=':memory:')

# Criar tabelas
con.execute("""
CREATE TABLE weather (
    city VARCHAR,         -- Nome da cidade
    temp_lo INTEGER,      -- Temperatura mínima
    temp_hi INTEGER,      -- Temperatura máxima
    prcp FLOAT,          -- Precipitação
    date DATE            -- Data da medição
);

CREATE TABLE cities (
    name VARCHAR,        -- Nome da cidade
    lat DECIMAL,        -- Latitude
    lon DECIMAL         -- Longitude
);
""")

print("✓ Tabelas 'weather' e 'cities' criadas com sucesso!")
print(con.fetchall())

✓ Tabelas 'weather' e 'cities' criadas com sucesso!
[]


## 4. Inserção de Dados (INSERT)

Existem diferentes formas de inserir dados em uma tabela:

### 4.1 Inserção Básica
Valores na ordem exata das colunas da tabela.

In [4]:
# Inserção básica - valores na ordem das colunas
con.execute("""
INSERT INTO weather VALUES ('San Francisco', 46, 50, 0.25, '1994-11-27');
""")
print("✓ Registro inserido (método básico)")
print(con.fetchall())

✓ Registro inserido (método básico)
[(1,)]


### 4.2 Inserção com Colunas Explícitas
Especificando quais colunas estamos preenchendo (mais seguro e legível).

In [5]:
# Inserção com colunas explícitas
con.execute("""
INSERT INTO weather (city, temp_lo, temp_hi, prcp, date)
VALUES ('San Francisco', 43, 57, 0.0, '1994-11-29');
""")
print("✓ Registro inserido (colunas explícitas)")
print(con.fetchall())

✓ Registro inserido (colunas explícitas)
[(1,)]


### 4.3 Inserção Omitindo Colunas
Colunas não especificadas recebem valor NULL.

In [6]:
# Inserção omitindo colunas (valores serão NULL)
con.execute("""
INSERT INTO weather (date, city, temp_hi, temp_lo)
VALUES ('1994-11-29', 'Hayward', 54, 37);
""")
print("✓ Registro inserido (com valores NULL em prcp)")
print(con.fetchall())

✓ Registro inserido (com valores NULL em prcp)
[(1,)]


### 4.4 Inserir Dados na Tabela Cities
Vamos adicionar algumas cidades para demonstrar junções posteriormente.

In [7]:
# Inserir dados de localização das cidades
con.execute("""
INSERT INTO cities VALUES ('San Francisco', 37.7749, -122.4194);
INSERT INTO cities VALUES ('Hayward', 37.6688, -122.0808);
""")
print("✓ Cidades inseridas na tabela 'cities'")
print(con.fetchall())

✓ Cidades inseridas na tabela 'cities'
[(1,)]


## 5. Consultas Básicas (SELECT)

O comando SELECT é usado para recuperar dados das tabelas.

### 5.1 Selecionar Todas as Colunas

In [8]:
# Selecionar todas as colunas usando *
resultado = con.execute("""
SELECT * FROM weather;
""")
print("Todos os registros de 'weather':")
print(resultado.fetchall())

Todos os registros de 'weather':
[('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29)), ('Hayward', 37, 54, None, datetime.date(1994, 11, 29))]


### 5.2 Selecionar Colunas Específicas
É uma boa prática especificar apenas as colunas necessárias.

In [9]:
# Selecionar colunas específicas
resultado = con.execute("""
SELECT city, temp_lo, temp_hi, prcp, date FROM weather;
""")
print("Colunas específicas:")
print(resultado.fetchall())

Colunas específicas:
[('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29)), ('Hayward', 37, 54, None, datetime.date(1994, 11, 29))]


## 6. Expressões e Cálculos

Podemos realizar cálculos diretamente nas queries SQL.

### 6.1 Calcular Temperatura Média
Usando operadores aritméticos e alias (AS) para nomear colunas calculadas.

In [10]:
# Calcular temperatura média
resultado = con.execute("""
SELECT 
    city, 
    temp_lo,
    temp_hi,
    (temp_hi + temp_lo) / 2 AS temp_avg, 
    date
FROM weather;
""")
print("Temperatura média calculada:")
print(resultado.fetchall())

Temperatura média calculada:
[('San Francisco', 46, 50, 48.0, datetime.date(1994, 11, 27)), ('San Francisco', 43, 57, 50.0, datetime.date(1994, 11, 29)), ('Hayward', 37, 54, 45.5, datetime.date(1994, 11, 29))]


### 6.2 Outros Cálculos
Exemplos adicionais de expressões aritméticas.

In [11]:
# Calcular amplitude térmica (diferença entre max e min)
resultado = con.execute("""
SELECT 
    city,
    temp_hi,
    temp_lo,
    (temp_hi - temp_lo) AS amplitude,
    date
FROM weather;
""")
print("Amplitude térmica:")
print(resultado.fetchall())

Amplitude térmica:
[('San Francisco', 50, 46, 4, datetime.date(1994, 11, 27)), ('San Francisco', 57, 43, 14, datetime.date(1994, 11, 29)), ('Hayward', 54, 37, 17, datetime.date(1994, 11, 29))]


## 7. Filtros com WHERE

A cláusula WHERE permite filtrar resultados com base em condições.

### 7.1 Filtro Simples com AND

In [12]:
# Filtrar por cidade E precipitação
resultado = con.execute("""
SELECT * FROM weather
WHERE city = 'San Francisco' AND prcp > 0.0;
""")
print("Dias chuvosos em San Francisco:")
print(resultado.fetchall())

Dias chuvosos em San Francisco:
[('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27))]


### 7.2 Filtros com Operadores de Comparação

In [13]:
# Filtrar por temperatura
resultado = con.execute("""
SELECT city, temp_hi, date FROM weather
WHERE temp_hi >= 50;
""")
print("Dias com temperatura máxima >= 50°F:")
print(resultado.fetchall())

Dias com temperatura máxima >= 50°F:
[('San Francisco', 50, datetime.date(1994, 11, 27)), ('San Francisco', 57, datetime.date(1994, 11, 29)), ('Hayward', 54, datetime.date(1994, 11, 29))]


### 7.3 Filtros com OR

In [14]:
# Filtrar com OR
resultado = con.execute("""
SELECT * FROM weather
WHERE city = 'San Francisco' OR city = 'Hayward';
""")
print("Registros de San Francisco ou Hayward:")
print(resultado.fetchall())

Registros de San Francisco ou Hayward:
[('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29)), ('Hayward', 37, 54, None, datetime.date(1994, 11, 29))]


## 8. Ordenação (ORDER BY)

ORDER BY organiza os resultados em ordem crescente (ASC) ou decrescente (DESC).

### 8.1 Ordenar por Uma Coluna

In [15]:
# Ordenar por cidade (ordem alfabética)
resultado = con.execute("""
SELECT * FROM weather ORDER BY city;
""")
print("Ordenado por cidade (ASC - padrão):")
print(resultado.fetchall())

Ordenado por cidade (ASC - padrão):
[('Hayward', 37, 54, None, datetime.date(1994, 11, 29)), ('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29))]


### 8.2 Ordenar por Múltiplas Colunas

In [16]:
# Ordenar por cidade e depois por temperatura mínima
resultado = con.execute("""
SELECT * FROM weather ORDER BY city, temp_lo;
""")
print("Ordenado por cidade e temp_lo:")
print(resultado.fetchall())

Ordenado por cidade e temp_lo:
[('Hayward', 37, 54, None, datetime.date(1994, 11, 29)), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29)), ('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27))]


### 8.3 Ordenação Decrescente

In [17]:
# Ordenar por temperatura máxima em ordem decrescente
resultado = con.execute("""
SELECT city, temp_hi, date FROM weather 
ORDER BY temp_hi DESC;
""")
print("Ordenado por temp_hi (DESC):")
print(resultado.fetchall())

Ordenado por temp_hi (DESC):
[('San Francisco', 57, datetime.date(1994, 11, 29)), ('Hayward', 54, datetime.date(1994, 11, 29)), ('San Francisco', 50, datetime.date(1994, 11, 27))]


## 9. Valores Únicos (DISTINCT)

DISTINCT remove duplicatas dos resultados.

### 9.1 Obter Cidades Únicas

In [18]:
# Obter lista de cidades únicas
resultado = con.execute("""
SELECT DISTINCT city FROM weather;
""")
print("Cidades únicas:")
print(resultado.fetchall())

Cidades únicas:
[('Hayward',), ('San Francisco',)]


### 9.2 DISTINCT com ORDER BY

In [19]:
# DISTINCT com ordenação
resultado = con.execute("""
SELECT DISTINCT city FROM weather 
ORDER BY city;
""")
print("Cidades únicas ordenadas:")
print(resultado.fetchall())

Cidades únicas ordenadas:
[('Hayward',), ('San Francisco',)]


## 10. Junções de Tabelas (JOINS)

JOINS combinam dados de múltiplas tabelas.

### 10.1 Junção Implícita (Cross Join com WHERE)
Método antigo, mas ainda funcional.

In [20]:
# Junção implícita
resultado = con.execute("""
SELECT * FROM weather, cities
WHERE city = name;
""")
print("Junção implícita (cross join + where):")
print(resultado.fetchall())

Junção implícita (cross join + where):
[('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27), 'San Francisco', Decimal('37.775'), Decimal('-122.419')), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29), 'San Francisco', Decimal('37.775'), Decimal('-122.419')), ('Hayward', 37, 54, None, datetime.date(1994, 11, 29), 'Hayward', Decimal('37.669'), Decimal('-122.081'))]


### 10.2 Junção com Colunas Específicas

In [21]:
# Especificando colunas na junção
resultado = con.execute("""
SELECT city, temp_lo, temp_hi, prcp, date, lon, lat
FROM weather, cities
WHERE city = name;
""")
print("Junção com colunas específicas:")
print(resultado.fetchall())

Junção com colunas específicas:
[('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27), Decimal('-122.419'), Decimal('37.775')), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29), Decimal('-122.419'), Decimal('37.775')), ('Hayward', 37, 54, None, datetime.date(1994, 11, 29), Decimal('-122.081'), Decimal('37.669'))]


### 10.3 INNER JOIN
Retorna apenas registros que têm correspondência em ambas as tabelas.

In [22]:
# INNER JOIN explícito
resultado = con.execute("""
SELECT 
    weather.city,
    weather.temp_lo,
    weather.temp_hi,
    cities.lat,
    cities.lon
FROM weather
INNER JOIN cities ON weather.city = cities.name;
""")
print("INNER JOIN:")
print(resultado.fetchall())

INNER JOIN:
[('San Francisco', 46, 50, Decimal('37.775'), Decimal('-122.419')), ('San Francisco', 43, 57, Decimal('37.775'), Decimal('-122.419')), ('Hayward', 37, 54, Decimal('37.669'), Decimal('-122.081'))]


### 10.4 LEFT OUTER JOIN
Retorna todos os registros da tabela da esquerda, mesmo sem correspondência.

In [23]:
# LEFT OUTER JOIN
resultado = con.execute("""
SELECT 
    weather.city,
    weather.temp_lo,
    weather.temp_hi,
    cities.lat,
    cities.lon
FROM weather
LEFT OUTER JOIN cities ON weather.city = cities.name;
""")
print("LEFT OUTER JOIN (inclui registros sem match):")
print(resultado.fetchall())

LEFT OUTER JOIN (inclui registros sem match):
[('San Francisco', 46, 50, Decimal('37.775'), Decimal('-122.419')), ('San Francisco', 43, 57, Decimal('37.775'), Decimal('-122.419')), ('Hayward', 37, 54, Decimal('37.669'), Decimal('-122.081'))]


## 11. Funções de Agregação

Funções que calculam valores agregados de um conjunto de linhas.

### 11.1 MAX - Valor Máximo

In [24]:
# Encontrar o valor máximo de temperatura mínima
resultado = con.execute("""
SELECT max(temp_lo) AS max_temp_lo FROM weather;
""")
print("Temperatura mínima mais alta:")
print(resultado.fetchall())

Temperatura mínima mais alta:
[(46,)]


### 11.2 MIN - Valor Mínimo

In [25]:
# Encontrar o valor mínimo de temperatura máxima
resultado = con.execute("""
SELECT min(temp_hi) AS min_temp_hi FROM weather;
""")
print("Temperatura máxima mais baixa:")
print(resultado.fetchall())

Temperatura máxima mais baixa:
[(50,)]


### 11.3 AVG - Média

In [26]:
# Calcular média de temperatura
resultado = con.execute("""
SELECT 
    avg(temp_lo) AS avg_temp_lo,
    avg(temp_hi) AS avg_temp_hi
FROM weather;
""")
print("Média das temperaturas:")
print(resultado.fetchall())

Média das temperaturas:
[(42.0, 53.666666666666664)]


### 11.4 COUNT - Contar Registros

In [27]:
# Contar registros
resultado = con.execute("""
SELECT 
    count(*) AS total_registros,
    count(prcp) AS registros_com_prcp
FROM weather;
""")
print("Contagem de registros:")
print(resultado.fetchall())

Contagem de registros:
[(3, 2)]


### 11.5 Subconsulta com Agregação
Encontrar a cidade com a temperatura mínima mais alta.

In [28]:
# Encontrar a cidade com a temperatura mínima mais alta
resultado = con.execute("""
SELECT city, temp_lo, date FROM weather
WHERE temp_lo = (SELECT max(temp_lo) FROM weather);
""")
print("Cidade com temp_lo mais alta:")
print(resultado.fetchall())

Cidade com temp_lo mais alta:
[('San Francisco', 46, datetime.date(1994, 11, 27))]


## 12. Agrupamento (GROUP BY)

GROUP BY agrupa linhas que têm valores iguais em colunas específicas.

### 12.1 Agrupar por Cidade

In [29]:
# Agrupar por cidade e calcular máximo
resultado = con.execute("""
SELECT 
    city, 
    max(temp_lo) AS max_temp_lo
FROM weather 
GROUP BY city;
""")
print("Temperatura mínima mais alta por cidade:")
print(resultado.fetchall())

Temperatura mínima mais alta por cidade:
[('San Francisco', 46), ('Hayward', 37)]


### 12.2 Múltiplas Agregações

In [30]:
# Múltiplas funções de agregação
resultado = con.execute("""
SELECT 
    city,
    count(*) AS num_registros,
    min(temp_lo) AS min_temp_lo,
    max(temp_hi) AS max_temp_hi,
    avg((temp_hi + temp_lo) / 2) AS avg_temp
FROM weather
GROUP BY city;
""")
print("Estatísticas por cidade:")
print(resultado.fetchall())

Estatísticas por cidade:
[('San Francisco', 2, 43, 57, 49.0), ('Hayward', 1, 37, 54, 45.5)]


## 13. Filtros em Grupos (HAVING)

HAVING filtra grupos após o GROUP BY (WHERE filtra antes).

### 13.1 Filtrar Grupos

In [31]:
# Filtrar grupos com HAVING
resultado = con.execute("""
SELECT city, max(temp_lo) AS max_temp_lo
FROM weather
GROUP BY city
HAVING max(temp_lo) < 40;
""")
print("Cidades com temp_lo máxima < 40:")
print(resultado.fetchall())

Cidades com temp_lo máxima < 40:
[('Hayward', 37)]


### 13.2 Combinando WHERE e HAVING
WHERE filtra linhas antes do agrupamento, HAVING filtra grupos.

In [32]:
# Combinar WHERE e HAVING
resultado = con.execute("""
SELECT city, max(temp_lo) AS max_temp_lo
FROM weather
WHERE city LIKE 'S%'  -- Filtro antes do agrupamento
GROUP BY city
HAVING max(temp_lo) < 50;  -- Filtro depois do agrupamento
""")
print("Cidades começando com 'S' e temp_lo máxima < 50:")
print(resultado.fetchall())

Cidades começando com 'S' e temp_lo máxima < 50:
[('San Francisco', 46)]


### 13.3 HAVING com Múltiplas Condições

In [33]:
# HAVING com múltiplas condições
resultado = con.execute("""
SELECT 
    city,
    count(*) AS num_registros,
    avg(temp_hi) AS avg_temp_hi
FROM weather
GROUP BY city
HAVING count(*) >= 1 AND avg(temp_hi) > 50;
""")
print("Cidades com múltiplos registros e temp_hi média > 50:")
print(resultado.fetchall())

Cidades com múltiplos registros e temp_hi média > 50:
[('San Francisco', 2, 53.5), ('Hayward', 1, 54.0)]


## 14. Atualização de Dados (UPDATE)

UPDATE modifica registros existentes.

### 14.1 Atualizar Uma Coluna
⚠️ **IMPORTANTE**: Sempre use WHERE para evitar atualizar todos os registros!

In [34]:
# Atualizar uma coluna específica
con.execute("""
UPDATE weather
SET prcp = 0.0
WHERE prcp IS NULL;
""")
print("✓ Valores NULL de prcp atualizados para 0.0")
print(con.fetchall())

# Verificar a atualização
resultado = con.execute("SELECT * FROM weather;")
print("\nDados após atualização:")
print(resultado.fetchall())

✓ Valores NULL de prcp atualizados para 0.0
[(1,)]

Dados após atualização:
[('San Francisco', 46, 50, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 43, 57, 0.0, datetime.date(1994, 11, 29)), ('Hayward', 37, 54, 0.0, datetime.date(1994, 11, 29))]


### 14.2 Atualizar Múltiplas Colunas

In [35]:
# Atualizar múltiplas colunas
con.execute("""
UPDATE weather
SET temp_hi = temp_hi - 2, 
    temp_lo = temp_lo - 2
WHERE date > '1994-11-28';
""")
print("✓ Temperaturas ajustadas para datas > '1994-11-28'")
print(con.fetchall())

# Verificar a atualização
resultado = con.execute("SELECT * FROM weather WHERE date > '1994-11-28';")
print("\nRegistros atualizados:")
print(resultado.fetchall())

✓ Temperaturas ajustadas para datas > '1994-11-28'
[(2,)]

Registros atualizados:
[('San Francisco', 41, 55, 0.0, datetime.date(1994, 11, 29)), ('Hayward', 35, 52, 0.0, datetime.date(1994, 11, 29))]


### 14.3 Atualização com Cálculo

In [36]:
# Atualização usando cálculo baseado em valores existentes
con.execute("""
UPDATE weather
SET temp_hi = temp_hi + 5
WHERE city = 'San Francisco';
""")
print("✓ Temperatura máxima aumentada em 5°F para San Francisco")
print(con.fetchall())

# Verificar
resultado = con.execute("SELECT * FROM weather WHERE city = 'San Francisco';")
print("\nRegistros de San Francisco:")
print(resultado.fetchall())

✓ Temperatura máxima aumentada em 5°F para San Francisco
[(2,)]

Registros de San Francisco:
[('San Francisco', 46, 55, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 41, 60, 0.0, datetime.date(1994, 11, 29))]


## 15. Exclusão de Dados (DELETE)

DELETE remove registros das tabelas.

### 15.1 Deletar Linhas Específicas
⚠️ **SEMPRE use WHERE** para especificar quais linhas deletar!

In [37]:
# Deletar linhas específicas
con.execute("""
DELETE FROM weather 
WHERE city = 'Hayward';
""")
print("✓ Registros de Hayward deletados")
print(con.fetchall())

# Verificar
resultado = con.execute("SELECT * FROM weather;")
print("\nRegistros restantes:")
print(resultado.fetchall())

✓ Registros de Hayward deletados
[(1,)]

Registros restantes:
[('San Francisco', 46, 55, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 41, 60, 0.0, datetime.date(1994, 11, 29))]


### 15.2 Deletar com Condições Múltiplas

In [38]:
# Deletar com múltiplas condições
con.execute("""
DELETE FROM weather
WHERE temp_hi < 50 AND prcp = 0.0;
""")
print("✓ Deletados registros com temp_hi < 50 e sem precipitação")
print(con.fetchall())

# Verificar
resultado = con.execute("SELECT * FROM weather;")
print("\nRegistros restantes:")
print(resultado.fetchall())

✓ Deletados registros com temp_hi < 50 e sem precipitação
[(0,)]

Registros restantes:
[('San Francisco', 46, 55, 0.25, datetime.date(1994, 11, 27)), ('San Francisco', 41, 60, 0.0, datetime.date(1994, 11, 29))]


### ⚠️ 15.3 CUIDADO: DELETE sem WHERE

**NUNCA execute DELETE sem WHERE em produção!**

Isso remove **TODAS** as linhas da tabela!

In [39]:
# ⚠️ PERIGO: Isso remove TODAS as linhas!
# Descomente apenas se tiver certeza:
# con.execute("DELETE FROM weather;")

print("⚠️ DELETE sem WHERE remove TODOS os registros!")
print("Execute com muito cuidado!")

# Verificar quantos registros temos
resultado = con.execute("SELECT count(*) FROM weather;")
print(f"\nRegistros atuais na tabela: {resultado.fetchall()}")

⚠️ DELETE sem WHERE remove TODOS os registros!
Execute com muito cuidado!

Registros atuais na tabela: [(2,)]


## 🎯 Resumo do Capítulo

Neste capítulo, exploramos:

1. ✅ **Instalação** do DuckDB
2. ✅ **Criação de tabelas** e estruturas
3. ✅ **Inserção de dados** (INSERT)
4. ✅ **Consultas básicas** (SELECT)
5. ✅ **Expressões e cálculos** em queries
6. ✅ **Filtros** com WHERE
7. ✅ **Ordenação** com ORDER BY
8. ✅ **Valores únicos** com DISTINCT
9. ✅ **Junções** (INNER JOIN, LEFT OUTER JOIN)
10. ✅ **Funções de agregação** (MAX, MIN, AVG, COUNT)
11. ✅ **Agrupamento** com GROUP BY
12. ✅ **Filtros de grupos** com HAVING
13. ✅ **Atualização** de dados (UPDATE)
14. ✅ **Exclusão** de dados (DELETE)

### Próximos Passos
No próximo capítulo, exploraremos instalação e configuração avançada do DuckDB!

## 🧹 Limpeza (Opcional)

Fechar a conexão quando terminar.

In [ ]:
# Fechar conexão
#con.close()
print("✓ Conexão fechada")

✓ Conexão fechada


In [41]:
con.execute("""
-- Atualizar múltiplas colunas
UPDATE weather
SET temp_hi = temp_hi - 2, temp_lo = temp_lo - 2
WHERE date > '1994-11-28';
""")
print(con.fetchall())

ConnectionException: Connection Error: Connection already closed!

## 14. Atualização de Dados (UPDATE)

Modificando registros existentes.

In [ ]:
con.execute("""
-- Cidades que começam com 'S'
SELECT city, max(temp_lo) FROM weather
WHERE city LIKE 'S%'
GROUP BY city
HAVING max(temp_lo) < 40;
""")
print(con.fetchall())

### Combinando WHERE e HAVING

In [ ]:
con.execute("""
-- Filtrar grupos
SELECT city, max(temp_lo) FROM weather
GROUP BY city
HAVING max(temp_lo) < 40;
""")
print(con.fetchall())

## 13. Filtros em Grupos (HAVING)

Filtrando resultados após agrupamento.

In [ ]:
con.execute("""
-- Agrupar por cidade
SELECT city, max(temp_lo) FROM weather GROUP BY city;
""")
print(con.fetchall())

## 12. Agrupamento (GROUP BY)

Agrupando dados para cálculos agregados.

In [ ]:
con.execute("""
-- Encontrar o valor máximo
SELECT max(temp_lo) FROM weather;

-- Encontrar a cidade com a temperatura mínima mais alta
SELECT city FROM weather
WHERE temp_lo = (SELECT max(temp_lo) FROM weather);
""")
print(con.fetchall())

## 11. Funções de Agregação

Funções para calcular valores agregados (MAX, MIN, AVG, etc.).

In [ ]:
con.execute("""
SELECT * FROM weather
LEFT OUTER JOIN cities ON weather.city = cities.name;
""")
print(con.fetchall())

### LEFT OUTER JOIN

In [ ]:
con.execute("""
SELECT * FROM weather
INNER JOIN cities ON weather.city = cities.name;
""")
print(con.fetchall())

### INNER JOIN

In [ ]:
con.execute("""
-- Junção implícita
SELECT * FROM weather, cities
WHERE city = name;

-- Especificando colunas
SELECT city, temp_lo, temp_hi, prcp, date, lon, lat
FROM weather, cities
WHERE city = name;
""")
print(con.fetchall())

## 10. Junções de Tabelas (JOINS)

Combinando dados de múltiplas tabelas.

In [ ]:
con.execute("""
-- Obter cidades únicas
SELECT DISTINCT city FROM weather;

-- DISTINCT com ordenação
SELECT DISTINCT city FROM weather ORDER BY city;
""")
print(con.fetchall())

## 9. Valores Únicos (DISTINCT)

Removendo duplicatas nos resultados.

In [ ]:
con.execute("""
-- Ordenar por uma coluna
SELECT * FROM weather ORDER BY city;

-- Ordenar por múltiplas colunas
SELECT * FROM weather ORDER BY city, temp_lo;
""")
print(con.fetchall())

## 8. Ordenação (ORDER BY)

Ordenando resultados.

In [ ]:
con.execute("""
-- Filtro simples
SELECT * FROM weather
WHERE city = 'San Francisco' AND prcp > 0.0;
""")
print(con.fetchall())

## 7. Filtros com WHERE

Filtrando resultados com condições.

In [ ]:
con.execute("""
-- Calcular temperatura média
SELECT city, (temp_hi + temp_lo) / 2 AS temp_avg, date
FROM weather;
""")
print(con.fetchall())

## 6. Expressões e Cálculos

Realizando cálculos em queries.

In [ ]:
con.execute("""
-- Selecionar todas as colunas
SELECT * FROM weather;

-- Selecionar colunas específicas
SELECT city, temp_lo, temp_hi, prcp, date FROM weather;
""")
print(con.fetchall())

## 5. Consultas Básicas (SELECT)

Selecionando dados das tabelas.

In [ ]:
con.execute("""
-- Inserção básica
INSERT INTO weather VALUES ('San Francisco', 46, 50, 0.25, '1994-11-27');

-- Inserção com colunas explícitas
INSERT INTO weather (city, temp_lo, temp_hi, prcp, date)
VALUES ('San Francisco', 43, 57, 0.0, '1994-11-29');

-- Inserção omitindo colunas (valores serão NULL)
INSERT INTO weather (date, city, temp_hi, temp_lo)
VALUES ('1994-11-29', 'Hayward', 54, 37);
""")
print(con.fetchall())

## 4. Inserção de Dados (INSERT)

Diferentes formas de inserir dados nas tabelas.

In [ ]:
# Conectar ao banco em memória
import duckdb
con = duckdb.connect(database=':memory:')

# Criar tabelas
con.execute("""
CREATE TABLE weather (
    city VARCHAR,
    temp_lo INTEGER,      -- temperatura mínima
    temp_hi INTEGER,      -- temperatura máxima
    prcp FLOAT,
    date DATE
);

CREATE TABLE cities (
    name VARCHAR,
    lat DECIMAL,
    lon DECIMAL
);
""")
print(con.fetchall())

## 3. Criação de Tabelas e Estruturas

Criando tabelas de exemplo para demonstrar operações SQL.

## 2. Imports e Configuração Inicial

Importando bibliotecas e criando conexão em memória.

In [ ]:
# Instalação de dependências
%pip install duckdb -q

## 1. Instalação de Dependências

Instalando o DuckDB para começar.